## LABELLING THE DATASET - PART 2

In [1]:
%pip install transformers vaderSentiment

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
from transformers import pipeline
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

c:\Users\Jonathan\Desktop\F20AA - Applied Text Analytics\CW1\F20AA_Grp5\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1. Labelling with RoBERTa based sentiment analysis model

In [ ]:
# Read the selected data
selected_data = pd.read_csv('../Data/selected_data.csv')

True

In [ ]:
# Initialize the sentiment analysis pipeline
sentiment_pipeline = pipeline("text-classification", 
                              model="cardiffnlp/twitter-roberta-base-sentiment-latest",
                              device=0) 

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


In [ ]:
# Extract the text column of selected_data as a list
reviews = selected_data["text"].tolist()

In [ ]:
# Calculate the sentiment of the each of the reviews
kwargs = {'padding':True,'truncation':True,'max_length':512}
results = sentiment_pipeline(reviews, **kwargs) 

In [ ]:
selected_data["roberta_label"] = [res["label"] for res in results]
selected_data["roberta_score"] = [res["score"] for res in results]

### Labelling with VADER (Lexicon and Rule-Based Model)

VADER was selected as it is specialized for sentiment analysis on social media comments.

In [ ]:
sentimentAnalyzer = SentimentIntensityAnalyzer()

In [ ]:
vader_label, vader_score = [], []

for review in reviews:
    # Calculate the sentiment of the review using VADER
    sentiment = sentimentAnalyzer.polarity_scores(review)
    vader_score.append(sentiment["compound"])
    
    if sentiment["compound"] >= 0.07:
        vader_label.append("positive")
    elif sentiment["compound"] <= -0.07:
        vader_label.append("negative")
    else:
        vader_label.append("neutral")


In [ ]:
# Add the VADER sentiment label and score to the selected_data DataFrame
selected_data["vader_label"] = vader_label
selected_data["vader_score"] = vader_score

In [ ]:
# Display the first few rows of the selected_data DataFrame
selected_data.head()

,post_id,subreddit,post_title,post_body,number_of_comments,readable_datetime,post_author,number_of_upvotes,query,text,comment_id,comment_body,comment_author,cosine_similarity,roberta_label,roberta_score,vader_label,vader_score


In [ ]:
# Save the selected_data DataFrame to a new CSV file
selected_data.to_csv('../Data/labelled_data.csv', index=False)